In [5]:
from src.dataloader import CompetingRiskDataset, DataLoader
from src.loss import total_loss
from src.model import DeepHit
from src.preprocess import preprocess_pipe
from src.train import plot_log, train
import torch
from torch.optim import Adam


dataset_transformed_train, dataset_transformed_val = preprocess_pipe(dataset_hf="Gabriel/synthetic_competing_risk")

training_data = CompetingRiskDataset(dataset_transformed_train)
validation_data = CompetingRiskDataset(dataset_transformed_val)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = DeepHit().to(device)
total_fn = total_loss

# hyperparameters
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 200
early_stopping_tol = 2
early_stopping_min_delta = 0.005
alpha = 0.3
sigma = 0.1
batch_train_size = 256
batch_val_size = 256
push_to_hub = True

training_loader = DataLoader(training_data, batch_size=batch_train_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_val_size, shuffle=True)

log_train, log_val = train(
    model,
    epochs,
    optimizer,
    total_fn,
    training_loader,
    validation_loader,
    device,
    early_stopping_tol,
    early_stopping_min_delta,
    alpha,
    sigma,
    push_to_hub,
)

plot_log(log_train, log_val)

FileNotFoundError: Unable to find '/home/gabriel/Desktop/deephit/.cache/train.parquet' at /home/gabriel/Desktop/deephit